In [5]:
! pip install tabula-py

  Using cached https://files.pythonhosted.org/packages/2a/60/15cee99f54273b78ae3c1846c1e85014975dbe503c09484c4e82b273f1b8/tabula_py-1.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f6/b1/ba5a96bccd3496241d8908164b9502a129156443cdd5acbdbf04a90b7a09/distro-1.3.0-py2.py3-none-any.whl


In [6]:
import tabula

In [9]:
! ls ../New_client1/Doc1/sample_tables.pdf

../New_client1/aa.pdf


In [ ]:
from tabula import read_pdf
df = read_pdf("../New_client1/Doc1/sample_tables.pdf", multiple_tables=False)

In [36]:
df

,Number of Coils,Number of Paperclips
0,5,"3, 5, 4"
1,10,"7, 8, 6"
2,15,"11, 10, 12"
3,20,"15, 13, 14"


In [23]:
read_pdf?

In [1]:

import pandas as pd 
import re
import sys  

#reload(sys)  
#sys.setdefaultencoding('iso-8859-1')



In [2]:
!pwd

/home/jovyan/Notebooks


In [75]:
########################read text file######################## 'Audit/NLP/test.txt', 'r'
with open('test.txt', 'r') as myfile:
    statement=myfile.readlines()

myfile.close()
#print(statement)


with open('test.txt', 'r') as myfile1:
    text=myfile1.read()

myfile1.close()
#print(text)


In [76]:
#########################Read dictionary######################## 
dic = pd.read_csv('test.csv',encoding='iso-8859-1',header=0)
#print(dic)
# Seperate text file to get Income Statement Block
Match_List  = []
for match in re.finditer('STATEMENT.*(\n|\r|\r\n).*(JUNE|DECEMBER) [0-9]*'.lower(), text.lower()):
    Match_List.append(match.span())

Income1 = Match_List[0][0]
Income2 = Match_List[1][0]

Income_statement = text[Income1:Income2]

#Income Statement In List Form 
Income_statement = Income_statement.split('\n') 
print(Income_statement)


['STATEMENT OF PROFIT OR LOSS AND OTHER COMPREHENSIVE INCOME', 'FOR THE YEAR ENDED 30 JUNE 2017', '', '', '', '                                                                                                  FY17             FY16', '                                                                             Note                 $’000            $’000', '', '', 'Interest income                                                                 1                    13                 17', 'Interest expense                                                                2                (1,937)          (4,050)', 'Net interest income                                                                              (1,924)          (4,033)', '', '', 'Other income                                                                    1               21,696           27,721', 'Fee and commission expense                                                      2                (1,148)          (3,045)', 'Ot

In [77]:
#Above Approach needs function so that the statments can get seperated automatically 

####################### Dictionary Mapping #######################

Income_line_item = []
for vocab in dic.Items:
    for statement in Income_statement:
        if vocab.lower() in statement.lower():
            Income_line_item.append(statement)
print(Income_line_item)

def accounting_number(n):
    if "," in n:
        n="".join(n.split(','))
        if n.isnumeric():
            n= float(n)
        elif n[0] =="(":
            n = - float(n[1:-1])
        elif "," in n:
            n=float("".join(n.split(',')))
    elif n.isnumeric():
        n= float(n)
    elif n[0] =="(":
        n = - float(n[1:-1])
    elif "," in n:
        n=float("".join(n.split(',')))
    elif n =='-':
        n= float(0)
    else:
        n
    
    return n 



['Interest income                                                                 1                    13                 17', 'Net interest income                                                                              (1,924)          (4,033)', 'Interest expense                                                                2                (1,937)          (4,050)', 'Net interest income                                                                              (1,924)          (4,033)', 'Other income                                                                    1               21,696           27,721', 'Fee and commission expense                                                      2                (1,148)          (3,045)', 'Other expenses                                                                  2                (5,628)        (11,728)', 'Loan impairment expense                                                         2                (2,346)                 -', 

In [78]:
def table_parsing(L):
    final_ray = []
    for i in L:
        number_list = []
        item_list =[]
        for n in i.split(): 
            if type(accounting_number(n)) == float:
                number_list.append(accounting_number(n))
            else:
                item_list.append(accounting_number(n))
        final_item =" ".join(item_list)
        if len(number_list)==3:
            number_list.insert(0, final_item)
            final_ray.append(number_list)
    
        elif len(number_list)==2:
            number_list.insert(0, 0)
            number_list.insert(0, final_item)
            final_ray.append(number_list)
    return final_ray



In [79]:

table_parsing(Income_line_item)

df = pd.DataFrame(table_parsing(Income_line_item), columns=['Items','Note','FY17','FY16'])
print(df)



                                        Items  Note     FY17     FY16
0                             Interest income   1.0     13.0     17.0
1                         Net interest income   0.0  -1924.0  -4033.0
2                            Interest expense   2.0  -1937.0  -4050.0
3                         Net interest income   0.0  -1924.0  -4033.0
4                                Other income   1.0  21696.0  27721.0
5                  Fee and commission expense   2.0  -1148.0  -3045.0
6                              Other expenses   2.0  -5628.0 -11728.0
7                     Loan impairment expense   2.0  -2346.0      0.0
8                           PROFIT BEFORE TAX   0.0  10650.0   8915.0
9                          Income tax expense   3.0  -3195.0  -2683.0
10                           PROFIT AFTER TAX   0.0   7455.0   6232.0
11  TOTAL COMPREHENSIVE INCOME FOR THE PERIOD   0.0   7455.0   6232.0
